# Model Training

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [31]:
import tensorflow as tf

import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [32]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [33]:
x_train.shape

(50000, 32, 32, 3)

In [34]:
x_test.shape

(10000, 32, 32, 3)

In [59]:
batchSize = 32
numClass = 10
epochs = 100

img_rows, img_cols = 32, 32

In [36]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

In [37]:
y_train = keras.utils.to_categorical(y_train, numClass)
y_test = keras.utils.to_categorical(y_test, numClass)

In [58]:
model = Sequential()
model.add(Conv2D(10, kernel_size=(3,3), \
                 activation='relu', input_shape=(32, 32, 3), \
                 padding='same', name='layer0'))
#model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(10, (3,3), activation='relu', padding='same', name='layer1'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(20, (3,3), activation='relu', padding='same', name='layer2'))
#model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(20, (3,3), activation='relu', padding='same', name='layer3'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(numClass, activation='softmax'))

In [60]:
model.compile(loss=keras.losses.categorical_crossentropy, \
              optimizer='adam', metrics=['accuracy'])

In [61]:
model.fit(x_train, y_train, batch_size=batchSize, epochs=epochs, \
          verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 10s 201us/step - loss: 1.6076 - accuracy: 0.4126 - val_loss: 1.2460 - val_accuracy: 0.5482
Epoch 2/100
50000/50000 [==============================] - 9s 187us/step - loss: 1.2717 - accuracy: 0.5416 - val_loss: 1.0679 - val_accuracy: 0.6142
Epoch 3/100
50000/50000 [==============================] - 9s 190us/step - loss: 1.1424 - accuracy: 0.5952 - val_loss: 1.0184 - val_accuracy: 0.6351
Epoch 4/100
50000/50000 [==============================] - 9s 188us/step - loss: 1.0648 - accuracy: 0.6226 - val_loss: 0.9401 - val_accuracy: 0.6732
Epoch 5/100
50000/50000 [==============================] - 9s 188us/step - loss: 1.0055 - accuracy: 0.6458 - val_loss: 0.9095 - val_accuracy: 0.6810
Epoch 6/100
50000/50000 [==============================] - 9s 189us/step - loss: 0.9630 - accuracy: 0.6608 - val_loss: 0.8535 - val_accuracy: 0.7035
Epoch 7/100
50000/50000 [==============================

In [64]:
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

Test loss:  0.744105731344223
Test accuracy:  0.7598999738693237


In [49]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer0 (Conv2D)              (None, 32, 32, 10)        760       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 16, 16, 10)        0         
_________________________________________________________________
layer1 (Conv2D)              (None, 16, 16, 20)        5020      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 8, 8, 20)          0         
_________________________________________________________________
layer2 (Conv2D)              (None, 8, 8, 20)          10020     
_________________________________________________________________
dropout_9 (Dropout)          (None, 8, 8, 20)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 1280)             

In [ ]:
pwd

'/content'

In [63]:
model.save('drive/My Drive/Colab Notebooks/cifar_keras.h5')

In [ ]:
import numpy as np

In [ ]:
def predict(idx, verbose=False):
    img = x_test[idx]
    label = y_test[idx]

    predict = model.predict_proba(np.expand_dims(img, axis=0))
    if verbose:
        print(f'Index: {idx}')
        cnt = 0
        for i in predict[0]:
            print('[{}] {:.2%}'.format(cnt, i))
            cnt += 1
        print('Predicted: ', predict[0].argmax())
        print('Label: ', label.argmax())
        plt.imshow(np.reshape(img, [28, 28]),interpolation='nearest', cmap='gray')
    return (predict[0].argmax(), label.argmax())

In [ ]:
# dictionary of mal-predicted indices 
wrong = {}

for i in range(x_test.shape[0]):
  pred = predict(i)
  if pred[0] != pred[1]:
    wrong[i] = (pred[0], pred[1])

In [ ]:
#choice = []
choice = open('drive/My Drive/Colab Notebooks/mnist_dropdown.txt', 'w')
for k, v in wrong.items():
    string = '%d) predicted: %d, label: %d' %(k, v[0], v[1])
    #choice.append(string)
    choice.write(string + '\n')